In [ ]:
import pandas as pd

import tqdm
from collections import defaultdict
import re
import matplotlib.pyplot as plt
import numpy as np
import gc
import mojimoji
import itertools

import category_encoders as ce

import matplotlib
import japanize_matplotlib
import seaborn as sns



# 所在地→緯度経度
import geocoder

In [ ]:
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")

In [ ]:
train.head(3)

In [ ]:
test.head(3)

In [ ]:
# 件数確認
train.shape, test.shape

# タイプミス修正

In [ ]:
train_fix = [
    [3334, "所在地", "東京都中央区晴海２丁目２－２－４２", "京都中央区晴海２丁目２－４２"],
    [7088, "所在地", "東京都大田区池上８丁目8-6-2", "東京都大田区池上８丁目6-2"],
    [7491, "面積", "5.83m2", "58.3m2"],
    [19363, "所在地", "東京都大田区池上８丁目8-6-2", "東京都大田区池上８丁目6-2"],
    [20231, "築年数", "520年5ヶ月", "20年5ヶ月"],
    [20427, "築年数", "1019年7ヶ月", "19年7ヶ月"],
    [20926, "面積", "430.1m2", "43.1m2"],
    [21285, "所在地", "東京都北区西ケ原３丁目西ヶ原３丁目", "東京都北区西ケ原３丁目"],
    [22249, "所在地", "東京都中央区晴海２丁目２－２－４２", "東京都中央区晴海２丁目２－４２"],
    [22883, "所在地", "東京都新宿区下落合２丁目2-1-17", "東京都新宿区下落合２丁目1-17"],
    [27198, "所在地", "東京都中央区晴海２丁目２－２－４２", "東京都中央区晴海２丁目２－４２"],
    [28140, "所在地", "東京都北区西ケ原１丁目西ヶ原１丁目", "東京都北区西ケ原１丁目"]
]

In [ ]:
for idx, col, prev, new in train_fix:
    print(idx, col, train.loc[idx, col], "->", new)
    train.loc[idx, col] = new

In [ ]:
train["賃料"].plot.hist(bins=30)

In [ ]:
train["賃料"].plot.hist(bins=30, logy=True)

In [ ]:
# 件数確認
train.shape, test.shape

# 前処理

In [ ]:
def parse_tab(text):
    if text != text:
        return set()
    text = mojimoji.zen_to_han(text, kana=False)
    text = text.replace("/", "")
    vals = set(text.split("\t"))
    vals -= {""}
    return vals

def encoding_tab(text, valset):
    valset = np.array(valset)
    vals = np.array(list(parse_tab(text)))
    if len(vals) == 0:
        return np.full(len(valset), 0, dtype=np.uint8)
    return np.isin(valset, vals).astype(np.uint8)

In [ ]:
train_and_test = pd.concat([train,test],axis=0)

# 所在地

In [ ]:
def get_address(df):
    location = df["所在地"].str.replace("東京都","").str.split("区", expand=True).rename(columns={0:"区", 1:"番地"})
    # 元のデータと結合し、不要になった所在地と詳細住所を削除
    df = pd.concat([df, location],axis=1)

    # 区情報の数値化(label encoding)
    labels, uniques = pd.factorize(df["区"])
    df["区"] = labels
    return df

In [ ]:
train_tmp = get_address(train_and_test)
train_tmp[["所在地", "区", "番地"]]

# 築年数

In [ ]:
def get_age_text(text):
    if text == "新築":
        return 0.0
    try:
        year_tmp = re.search(r"\d+年", text)
        month_tmp = re.search(r"\d+ヶ月", text)
    except:
        return np.nan
    if year_tmp:
        year = int(year_tmp.group()[:-1])
    else:
        year = 0
    if month_tmp:
        month = int(month_tmp.group()[:-2])
    else:
        month = 0
    return year + month / 12

def get_age(df):
    values = df["築年数"].values
    ages = []
    for text in values:
        ages.append(get_age_text(text))
    if len(ages) != len(df):
        return "築年数error"
    df["age"] = ages
    df["age_round"] = df["age"].round()
    
    df = df.drop("築年数", axis=1)
    
    return df

In [ ]:
train_tmp = get_age(train_and_test)
train_tmp[["age"]]

# 所在階

In [ ]:
def get_floor(df):
    # NaN埋めし、「地下」を「-」に変換
    df["所在階"] = df["所在階"].fillna("不明").str.replace("地下","-")

    # まずは「X」階／「Y」階建の表記になっているレコードを変換
    train_and_test_match = df[df["所在階"].str.match("[\-0-9]+階／[\-0-9]+階建")].copy()
    floors_match = train_and_test_match["所在階"].str.extract("(?P<floor>[\-0-9]+)階／(?P<all_floor>[\-0-9]+)階建")
    train_and_test_match = pd.concat([train_and_test_match,floors_match],axis=1)

    # 上の表記になっていない(すなわち、どちらかの情報が欠損している)レコードを抽出
    train_and_test_unmatch = df[~df["所在階"].str.match("[\-0-9]+階／[\-0-9]+階建")].copy()

    # 「Y」階建だけの表記になっているレコードを変換
    train_and_test_unmatch_0 = train_and_test_unmatch[train_and_test_unmatch["所在階"].str.contains("階建")].copy()
    floors_unmatch_0 = train_and_test_unmatch_0["所在階"].str.extract("(?P<all_floor>[\-0-9]+)階建")
    train_and_test_unmatch_0 = pd.concat([train_and_test_unmatch_0,floors_unmatch_0],axis=1)

    # 「X」階だけの表記になっているレコードを変換
    train_and_test_unmatch_1 = train_and_test_unmatch[~train_and_test_unmatch["所在階"].str.contains("階建")].copy()
    floors_unmatch_1 = train_and_test_unmatch_1["所在階"].str.extract("(?P<floor>[\-0-9]+)階")
    train_and_test_unmatch_1 = pd.concat([train_and_test_unmatch_1,floors_unmatch_1],axis=1)

    df = pd.concat([train_and_test_match,train_and_test_unmatch_0,train_and_test_unmatch_1],axis=0)
    # 「X」階、「Y」階建どちらの情報もNaNになっているレコードは1つしかないはず。
    # 2つ以上あると変換ミスが起きていることになるので、1つしかないことを確認
    print(len(df[(df["floor"].isnull())&(df["all_floor"].isnull())]))

    df = df.drop("所在階", axis=1)

    # 数値型に変換する。NaNが入っていると変換できないので、一度適当な値で埋めて変換する
    df["floor"] = df["floor"].fillna(99999)
    df["all_floor"] = df["all_floor"].fillna(99999)
    df = df.astype({"floor":"int16","all_floor":"int16"})
    #「X」階 = floor の値が負になっているレコードを一律-1に設定
    df.loc[df["floor"]<0,"floor"] = -1
    df.loc[df["all_floor"]<0,"all_floor"] = -1
    # 99999をNaNに戻す
    df["floor"] = df["floor"].replace(99999,np.nan)
    df["all_floor"] = df["all_floor"].replace(99999,np.nan)
    
    return df

In [ ]:
train_tmp = get_floor(train_and_test)
train_tmp[["floor", "all_floor"]]

# 契約期間

In [ ]:
def get_contr(df):
    # NaNがあると動かない処理が多くあるので、最初にNaNを適当な値で埋める
    df["契約期間"] = df["契約期間"].fillna("不明")

    # 定期借家か否かのフラグ (True/Falseで出てくるので1/0に変換)
    df["定期借家フラグ"] = df["契約期間"].str.contains("定期借家").map({False:0,True:1})

    # まずは「X」年「Yヶ月」間 表記になっているレコードのX, Yを抽出
    train_and_test_keiyaku1 = df[df["契約期間"].str.match("[0-9]+年[0-9+]ヶ月間")].copy()
    keiyaku_year_month = train_and_test_keiyaku1["契約期間"].str.extract("(?P<契約_year>[0-9]+)年(?P<契約_month>[0-9]+)ヶ月間")
    keiyaku_year_month = keiyaku_year_month.astype({"契約_year":"int8","契約_month":"int8"})
    # 月を年に換算して合算
    keiyaku_year_month["keiyaku_length"] = keiyaku_year_month["契約_year"]+(keiyaku_year_month["契約_month"]/12.0)
    # 元のデータフレームのidと結合(idはマージ用)
    train_and_test_keiyaku1 = pd.concat([train_and_test_keiyaku1[["id"]],keiyaku_year_month[["keiyaku_length"]]],axis=1)

    # 「X」年「Yヶ月」間 表記になっておらず、かつ「A年B月まで」表記にもなっていないレコードを抽出
    train_and_test_keiyaku2 = df[(~df["契約期間"].str.match("[0-9]+年[0-9+]ヶ月間"))&(~df["契約期間"].str.contains("まで"))].copy()

    # 「X」年間 表記になっているレコードを変換
    train_and_test_keiyaku2_year = train_and_test_keiyaku2[train_and_test_keiyaku2["契約期間"].str.match("[0-9]+年間")].copy()
    keiyaku_year = train_and_test_keiyaku2_year["契約期間"].str.extract("(?P<keiyaku_length>[0-9]+)年間")
    keiyaku_year = keiyaku_year.astype({"keiyaku_length":"int8"})
    train_and_test_keiyaku2_year = pd.concat([train_and_test_keiyaku2_year[["id"]],keiyaku_year],axis=1)
    # 「Y」ヶ月間 表記になっているレコードを変換
    train_and_test_keiyaku2_month = train_and_test_keiyaku2[train_and_test_keiyaku2["契約期間"].str.match("[0-9]+ヶ月間")].copy()
    keiyaku_month = train_and_test_keiyaku2_month["契約期間"].str.extract("(?P<keiyaku_length>[0-9]+)ヶ月間")
    keiyaku_month = keiyaku_month.astype({"keiyaku_length":"float32"})
    # 月を年に換算
    keiyaku_month["keiyaku_length"] /= 12.0
    train_and_test_keiyaku2_month = pd.concat([train_and_test_keiyaku2_month[["id"]],keiyaku_month],axis=1)

    # 「A年B月まで」表記にもなっているレコードを抽出 & 変換
    train_and_test_keiyaku3 = df[df["契約期間"].str.contains("まで")].copy()
    keiyaku_until = train_and_test_keiyaku3["契約期間"].str.extract("(?P<年>[0-9]+)年(?P<月>[0-9]+)月まで")
    keiyaku_until = keiyaku_until.astype({"年":"int16","月":"int8"})
    # 2019年8月を起点に引き算した後に年換算
    keiyaku_until["keiyaku_length"] = ((keiyaku_until["年"]-2019)*12+(keiyaku_until["月"]-8))/12.0
    train_and_test_keiyaku3 = pd.concat([train_and_test_keiyaku3[["id"]],keiyaku_until[["keiyaku_length"]]],axis=1)

    # 加工したデータフレームを結合
    train_and_test_keiyaku = pd.concat([train_and_test_keiyaku1,train_and_test_keiyaku2_year,train_and_test_keiyaku2_month,train_and_test_keiyaku3],axis=0)
    # 元のデータフレームにマージ
    df = df.merge(train_and_test_keiyaku,on="id",how="left")
    df = df.drop("契約期間", axis=1)
    
    return df

In [ ]:
train_tmp = get_contr(train_and_test)
train_tmp[["keiyaku_length", "定期借家フラグ"]]

# 間取り

In [ ]:
def get_plan(df):
    # ↑で確認した限り、必ず1文字目が部屋数なので1文字目を取得
    df["部屋数"] = df["間取り"].map(lambda x:int(x[0]))

    # K, D, L, Sのあるなしを取得
    df["K有無"] = df["間取り"].str.contains("K").astype(int)
    df["D有無"] = df["間取り"].str.contains("D").astype(int)
    df["L有無"] = df["間取り"].str.contains("L").astype(int)
    df["S有無"] = df["間取り"].str.contains("S").astype(int)
    df = df.drop("間取り", axis=1)

    return df

In [ ]:
train_tmp = get_plan(train_and_test)
train_tmp[["部屋数", "K有無", "D有無", "L有無", "S有無"]]

# 駐輪場、駐車場

In [ ]:
def parse_park(text):
    #無し→評価低、あり→普通、料金つき→優先度たかめ、料金入れるのはないデータもあるから微妙？
    if text != text:
        return np.full(3, 0, dtype=np.uint8)
    text = mojimoji.zen_to_han(text, kana=False)
    text = text.replace("\t", "")
    keys = re.findall(r"駐輪場|バイク置き場|駐車場", text)
    vals = re.split(r"駐輪場|バイク置き場|駐車場", text)[1:]
    result = np.zeros(3, dtype=np.uint8)
    columns = ["駐輪場", "バイク置き場", "駐車場"]
    for i, v in enumerate(vals):
        if "無" in v:
            result[columns.index(keys[i])] = 0
        else:
            price = re.search(r"\d+円", v)
            if price:
                result[columns.index(keys[i])] = 2
#                 result[key.index(keys[i])] = price
            elif "空有" in v:
                result[columns.index(keys[i])] = 1
    return result


def get_park(df):
    vals = df["駐車場"].values
    df_tmp = []
    for i, val in enumerate(vals):
        df_tmp.append(parse_park(val))
    df_tmp = pd.DataFrame(df_tmp, index=df.index,
                          columns=["park_bike", "park_motor", "park_car"])
    df = pd.concat([df, df_tmp], axis=1)
    df = df.drop("駐車場", axis=1)
    return df

In [ ]:
train_tmp = get_park(train_and_test)
train_tmp[["park_bike", "park_motor", "park_car"]]

# 面積

In [ ]:
def get_area(df):
    df["area"] = df["面積"].str.replace("m2", "").astype(float)
    df["area_round"] = df["area"].round()
    df = df.drop("面積", axis=1)
    return df

# 風呂・トイレ

In [ ]:
def get_bus_toilet(df):
    columns = ["トイレなし", "共同トイレ", "専用バス", "脱衣所", "バス・トイレ別",
              "専用トイレ", "シャワー", "洗面台独立", "共同バス", "浴室乾燥機",
              "追焚機能", "温水洗浄便座", "バスなし"]
    vals = df["バス・トイレ"].values
    df_tmp = []
    for i, val in enumerate(vals):
        df_tmp.append(encoding_tab(val, columns))
    df_tmp = pd.DataFrame(df_tmp, index=df.index, columns=columns)
    df = pd.concat([df, df_tmp], axis=1)
    df = df.drop("バス・トイレ", axis=1)
    
    return df

# 周辺環境

In [ ]:
def encode_env(text, valset):
    # 近いほどいいので、適当にでかい値を入れておく
    items = parse_tab(text)
    if len(items) == 0:
        return np.full(len(valset), 99999)
    dists = np.full(len(valset), 99999)
    for item in items:
        place = re.search(r"【.{1,9}】", item).group()
        idx = valset.index(place)
        dist = int(re.search(r"\d+", item).group())
        if dist < dists[idx]:
            dists[idx] = dist
    return dists

def get_env(df):
    columns = ["【銀行】", "【コンビニ】", "【小学校】", "【総合病院】", "【大学】",
              "【図書館】", "【デパート】", "【公園】", "【幼稚園・保育園】",
              "【コインパーキング】", "【クリーニング】", "【レンタルビデオ】",
              "【学校】", "【ドラッグストア】", "【月極駐車場】", "【スーパー】",
              "【病院】", "【飲食店】", "【郵便局】"]
    vals = df["周辺環境"].values
    df_tmp = []
    for i, val in enumerate(vals):
        df_tmp.append(encode_env(val, columns))
    df_tmp = pd.DataFrame(df_tmp, index=df.index, columns=columns)
    df = pd.concat([df, df_tmp], axis=1)
    df = df.drop("周辺環境", axis=1)
    
    return df

In [ ]:
val = ["【デパート】",
       "【総合病院】",
       "【図書館】",
       "【コンビニ】",
       "【公園】",
       "【ドラッグストア】",
       "【銀行】",
       "【月極駐車場】",
       "【大学】",
       "【クリーニング】",
       "【幼稚園・保育園】",
       "【レンタルビデオ】",
       "【スーパー】",
       "【学校】",
       "【小学校】",
       "【コインパーキング】",
       "【病院】",
       "【飲食店】",
       "【郵便局】",
      ]
train_tmp = get_env(train_and_test)
train_tmp[val]

# 室内設備

In [ ]:
def get_interior(df):
    columns = ["バルコニー", "井戸", "ルーフバルコニー", "ガス暖房", "室内洗濯機置場",
              "汲み取り", "シューズボックス", "エレベーター", "室外洗濯機置場", "エアコン付",
              "バリアフリー", "プロパンガス", "浄化槽", "床暖房", "ロフト付き", "フローリング",
              "専用庭", "石油暖房", "3面採光", "クッションフロア", "公営水道", "ガスその他",
              "出窓", "冷房", "水道その他", "防音室", "都市ガス", "下水", "オール電化",
              "二世帯住宅", "ウォークインクローゼット", "敷地内ごみ置き場", "トランクルーム",
              "24時間換気システム", "排水その他", "タイル張り", "2面採光", "二重サッシ",
              "床下収納", "地下室", "洗濯機置場なし", "ペアガラス"]
    vals = df["室内設備"].values
    df_tmp = []
    for i, val in enumerate(vals):
        df_tmp.append(encoding_tab(val, columns))
    df_tmp = pd.DataFrame(df_tmp, index=df.index, columns=columns)
    df = pd.concat([df, df_tmp], axis=1)
    df = df.drop("室内設備", axis=1)
    
    return df

In [ ]:
val = ["出窓",
 "水道その他",
 "専用庭",
 "下水",
 "シューズボックス",
 "排水その他",
 "室外洗濯機置場",
 "クッションフロア",
 "二重サッシ",
 "バルコニー",
 "エレベーター",
 "敷地内ごみ置き場",
 "床下収納",
 "二世帯住宅",
 "室内洗濯機置場",
 "床暖房",
 "2面採光",
 "ルーフバルコニー",
 "公営水道",
 "井戸",
 "24時間換気システム",
 "ガス暖房",
 "浄化槽",
 "バリアフリー",
 "フローリング",
 "ペアガラス",
 "ガスその他",
 "洗濯機置場なし",
 "エアコン付",
 "防音室",
 "ウォークインクローゼット",
 "オール電化",
 "汲み取り",
 "プロパンガス",
 "石油暖房",
 "3面採光",
 "ロフト付き",
 "地下室",
 "トランクルーム",
 "冷房",
 "都市ガス",
 "タイル張り"
      ]

train_tmp = get_interior(train_and_test)
train_tmp[val]

In [ ]:
def get_kitchen(df):
    columns = ["ガスコンロ", "冷蔵庫あり", "コンロ設置可(コンロ4口以上)", "カウンターキッチン",
              "IHコンロ", "コンロ設置可(コンロ1口)", "コンロ設置可(コンロ3口)", "コンロ3口",
              "電気コンロ", "コンロ設置可(口数不明)", "L字キッチン", "独立キッチン", "給湯",
              "コンロ4口以上", "コンロ2口", "システムキッチン", "コンロ1口", "コンロ設置可(コンロ2口)"]
    vals = df["キッチン"].values
    df_tmp = []
    for i, val in enumerate(vals):
        df_tmp.append(encoding_tab(val, columns))
    df_tmp = pd.DataFrame(df_tmp, index=df.index, columns=columns)
    df = pd.concat([df, df_tmp], axis=1)
    df = df.drop("キッチン", axis=1)

    return df

In [ ]:
val = ["ガスコンロ", "冷蔵庫あり", "コンロ設置可(コンロ4口以上)", "カウンターキッチン",
              "IHコンロ", "コンロ設置可(コンロ1口)", "コンロ設置可(コンロ3口)", "コンロ3口",
              "電気コンロ", "コンロ設置可(口数不明)", "L字キッチン", "独立キッチン", "給湯",
              "コンロ4口以上", "コンロ2口", "システムキッチン", "コンロ1口", "コンロ設置可(コンロ2口)"]

In [ ]:
train_tmp = get_kitchen(train_and_test)
train_tmp[val]

In [ ]:
def get_tv(df):
    columns = ["BSアンテナ", "光ファイバー", "CSアンテナ", "高速インターネット", "CATV",
              "有線放送", "インターネット対応", "インターネット使用料無料"]
    vals = df["放送・通信"].values
    df_tmp = []
    for i, val in enumerate(vals):
        df_tmp.append(encoding_tab(val, columns))
    df_tmp = pd.DataFrame(df_tmp, index=df.index, columns=columns)
    df = pd.concat([df, df_tmp], axis=1)
    df = df.drop("放送・通信", axis=1)
    
    return df

In [ ]:
val = ["BSアンテナ", "光ファイバー", "CSアンテナ", "高速インターネット", "CATV",
              "有線放送", "インターネット対応", "インターネット使用料無料"]

In [ ]:
train_tmp = get_tv(train_and_test)
train_tmp[val]

In [ ]:
def get_label(df, label):
    labels, uniques = pd.factorize(df[label])
    df[label] = labels
    return df

In [ ]:
def get_access(df):
    mottime = df["アクセス"].str.split("\t", expand=True).iloc[:,2:12:4]
    mottime1 = mottime[2].str.extract("徒歩(?P<徒歩_time1>[0-9]+)分")
    mottime2 = mottime[6].str.extract("徒歩(?P<徒歩_time2>[0-9]+)分")
    mottime3 = mottime[10].str.extract("徒歩(?P<徒歩_time3>[0-9]+)分")
    mottime = pd.concat([mottime1, mottime2, mottime3],axis=1).fillna(0)
    
    railway = pd.DataFrame(df["アクセス"].str.split("\t", expand=True)[0])
    railway.columns = ["railway"]

    count_encoder = ce.CountEncoder(cols=["railway"])
    railway = count_encoder.fit_transform(railway)
    
    df = pd.concat([df, mottime, railway], axis=1)
#     df = df.drop("アクセス", axis=1)
    return df

In [ ]:
train_and_test["アクセス"].str.split("\t", expand=True).iloc[:,2:12:4]

In [ ]:
train_tmp = get_access(train_and_test)
train_tmp

In [ ]:
def preprocess(df, drop_column=False):
    """
    特徴量を追加したり数値の形にする
    """
    print(f"start:{df.shape}")
    
    # 所在地
    df = get_address(df)
    print(f"所在地：{df.shape}")
    
    # 築年数
    df = get_age(df)
    print(f"築年数：{df.shape}")
    
    # 面積
    df = get_area(df)
    print(f"面積：{df.shape}")
    
    # バス・トイレ
    df = get_bus_toilet(df)
    print(f"バス・トイレ：{df.shape}")
    
    # 駐車場
    df = get_park(df)
    print(f"駐車場：{df.shape}")
    
    # 間取り
    df = get_plan(df)
    print(f"間取り：{df.shape}")
    
    # 契約期間
    df = get_contr(df)
    print(f"契約期間：{df.shape}")
    
    # 所在階
    df = get_floor(df)
    print(f"所在階：{df.shape}")
    
    # 周辺環境
    df = get_env(df)
    print(f"周辺環境：{df.shape}")
    
    # 室内設備
    df = get_interior(df)
    print(f"室内設備：{df.shape}")
    
    # キッチン
    df = get_kitchen(df)
    print(f"キッチン：{df.shape}")
    
    # TV
    df = get_tv(df)
    print(f"TV：{df.shape}")
    
    # アクセス
    df = get_access(df)
    print(f"アクセス：{df.shape}")
    
    # 方角、建物構造
    df = get_label(df, "方角")
    print(f"方角：{df.shape}")
    df = get_label(df, "建物構造")
    print(f"建物構造：{df.shape}")
    
    return df

In [ ]:
train_and_test = preprocess(train_and_test)

In [ ]:
train = train_and_test[train_and_test["賃料"].notnull()].copy().reset_index(drop=True)
test = train_and_test[train_and_test["賃料"].isnull()].copy().reset_index(drop=True)

print(train.shape,test.shape)

In [ ]:
train.to_csv("data/train_processed.csv", index=False)
test.to_csv("data/test_processed.csv", index=False)

# 四則演算特徴量作成

In [ ]:
cols = train_and_test.columns
cols = [
    "age",
    "age_round",
    "部屋数",
    "area",
    "area_round",
    "keiyaku_length",
    "floor",
    "all_floor",
]

In [ ]:
def add_sub_mul_div(df, col1, col2):
    df[f"{col1}+{col2}"] = df[col1] + df[col2]
    df[f"{col1}-{col2}"] = df[col1] - df[col2]
    df[f"{col1}*{col2}"] = df[col1] * df[col2]
    df[f"{col1}/{col2}"] = df[col1] / df[col2]
    return df

In [ ]:
for c in itertools.combinations(cols, 2):
    col1, col2 = list(c)
    train_and_test = add_sub_mul_div(train_and_test, col1, col2)

# Groupby特徴量作成

In [ ]:
cols = train_and_test.columns
cols = [
    "方角",
    "建物構造",
    "age_round",
    "区",
    "area_round",
    "keiyaku_length",
    "floor",
    "徒歩_time1",
]

In [ ]:
def make_groupby_from(df, groupby_col, agg_func):
    groupby_df = df.groupby(groupby_col).agg({"賃料":agg_func}).reset_index().fillna(0)
    agg_func_column = [f"{agg}_{"&".join(groupby_col)}" for agg in agg_func]
    groupby_df.columns = groupby_col + agg_func_column
    return groupby_df

In [ ]:
agg_func = ["mean", "median", "min", "max", "std"]

for c in itertools.combinations(cols, 1):
    groupby_col = list(c)
    groupby_df = make_groupby_from(train_and_test, groupby_col, agg_func)
    groupby_df[f"max-min_{"&".join(groupby_col)}"] = groupby_df[f"max_{"&".join(groupby_col)}"] - groupby_df[f"min_{"&".join(groupby_col)}"]
    train_and_test = pd.merge(train_and_test, groupby_df, on=groupby_col, how="left")
print("----------------1 done----------------")
# for c in itertools.combinations(cols, 2):
#     groupby_col = list(c)
#     groupby_df = make_groupby_from(train_and_test, groupby_col, agg_func)
#     groupby_df[f"max-min_{"&".join(groupby_col)}"] = groupby_df[f"max_{"&".join(groupby_col)}"] - groupby_df[f"min_{"&".join(groupby_col)}"]
#     train_and_test = pd.merge(train_and_test, groupby_df, on=groupby_col, how="left")
# print("----------------2 done----------------")
# for c in itertools.combinations(cols, 3):
#     groupby_col = list(c)
#     groupby_df = make_groupby_from(train_and_test, groupby_col, agg_func)
#     groupby_df[f"max-min_{"&".join(groupby_col)}"] = groupby_df[f"max_{"&".join(groupby_col)}"] - groupby_df[f"min_{"&".join(groupby_col)}"]
#     train_and_test = pd.merge(train_and_test, groupby_df, on=groupby_col, how="left")
# print("----------------3 done----------------")

In [ ]:
train = train_and_test[train_and_test["賃料"].notnull()].copy().reset_index(drop=True)
test = train_and_test[train_and_test["賃料"].isnull()].copy().reset_index(drop=True)
del train_and_test

train.to_csv("data/train_processed_add_groupby.csv", index=False)
test.to_csv("data/test_processed_add_groupby.csv", index=False)

gc.collect()

In [ ]:
print(train.shape,test.shape)